# Этап 1: Подготовка данных

#### Загрузка данных

In [3]:
# Импорт стандартных библиотек
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('sp500_historical_data.csv')

#### Преобразование столбца даты

In [6]:
data['Date'] = pd.to_datetime(data['Date'])

#### Удаление данных за выходные дни

In [8]:
data = data[data['Date'].dt.weekday < 5]

#### Определение полного диапазона дат

In [10]:
full_date_range = pd.date_range(start=data['Date'].min(), end=data['Date'].max(), freq='B')  # 'B' обозначает бизнес-день

#### Проверка наличия данных по каждому тикеру

In [12]:
data_pivot = data.pivot_table(index='Date', columns='Ticker', values='Adj Close')
missing_percent = data_pivot.isnull().mean() * 100
tickers_to_remove = missing_percent[missing_percent > 5].index.tolist()
clean_data = data[~data['Ticker'].isin(tickers_to_remove)]

#### Проверка на дупликаты

In [14]:
clean_data = clean_data.drop_duplicates()

#### Обработка подозрительно низкого объема торгов

In [16]:
volume_threshold = clean_data['Volume'].quantile(0.01)
low_volume_data = clean_data[clean_data['Volume'] <= volume_threshold]
low_volume_data.to_csv('low_volume_records.csv', index=False)

#### Проверка размерности данных

In [18]:
data_pivot_clean = clean_data.pivot_table(index='Date', columns='Ticker', values='Adj Close')
missing_dates_per_ticker = data_pivot_clean.isnull().sum()
tickers_with_missing_dates = missing_dates_per_ticker[missing_dates_per_ticker > 0].index.tolist()
clean_data = clean_data[~clean_data['Ticker'].isin(tickers_with_missing_dates)]

#### Сбор информации о секторах

In [20]:
# Установка yfinance при необходимости
# !pip install yfinance

import yfinance as yf

# Получение списка уникальных тикеров
tickers = data['Ticker'].unique()

# Инициализация словаря для хранения информации о секторах
ticker_sectors = {}

# Получение информации о секторах для каждого тикера
for ticker in tickers:
    try:
        stock_info = yf.Ticker(ticker).info
        sector = stock_info.get('sector', 'Unknown')
        ticker_sectors[ticker] = sector
    except Exception as e:
        print(f"Не удалось получить данные для {ticker}: {e}")
        ticker_sectors[ticker] = 'Unknown'

# Создание DataFrame из словаря
sectors_df = pd.DataFrame(list(ticker_sectors.items()), columns=['Ticker', 'Sector'])

# Объединение информации о секторах с основными данными
data = pd.merge(data, sectors_df, on='Ticker', how='left')

#### Сохранение очищенных данных

In [22]:
clean_tickers = clean_data['Ticker'].unique()
clean_data.to_csv('cleaned_sp500_data.csv', index=False)

In [52]:
import os

# Для разделения данных и предобработки
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Для построения модели LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

# Для генетического алгоритма
from deap import base, creator, tools, algorithms

# Для оценки модели
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Для визуализации
import matplotlib.pyplot as plt
%matplotlib inline

# Дополнительные библиотеки
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

#### Загрузка очищенных данных

In [25]:
# Загрузка очищенных данных
data = pd.read_csv('cleaned_sp500_data.csv')

# Преобразование столбца 'Date' в формат datetime
data['Date'] = pd.to_datetime(data['Date'])

# Этап 2: Построение и оптимизация модели LSTM с помощью GA

#### Подготовка данных

In [56]:
# Определение параметров
TIME_STEPS = 30  # Длина последовательности для входа в LSTM

# Инициализация словарей для хранения результатов
models = {}
scalers = {}
volatility = {}
performance = {}

#### Установка random для воспроизведения

In [59]:
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

#### Определение функций для подготовки данных

In [62]:
def create_sequences(data, time_steps=TIME_STEPS):
    X = []
    y = []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps)])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

#### Определение функции оценки модели

In [65]:
import tensorflow.keras.backend as K

def evaluate_model(individual, X_train, y_train, X_val, y_val):
    # Unpack hyperparameters
    n_layers = individual[0]
    n_neurons = individual[1]
    dropout_rate = individual[2]
    learning_rate = individual[3]

    # Build the model
    K.clear_session()
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))

    if n_layers == 1:
        # Single LSTM layer
        model.add(LSTM(units=n_neurons, return_sequences=False))
        model.add(Dropout(dropout_rate))
    else:
        # First LSTM layer
        model.add(LSTM(units=n_neurons, return_sequences=True))
        model.add(Dropout(dropout_rate))
        # Middle LSTM layers
        for _ in range(n_layers - 2):
            model.add(LSTM(units=n_neurons, return_sequences=True))
            model.add(Dropout(dropout_rate))
        # Last LSTM layer
        model.add(LSTM(units=n_neurons, return_sequences=False))
        model.add(Dropout(dropout_rate))

    model.add(Dense(1))

    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='mean_squared_error', optimizer=optimizer)

    # Train the model
    history = model.fit(X_train, y_train, epochs=5, batch_size=32,
                        validation_data=(X_val, y_val), verbose=0)

    # Evaluate validation loss
    val_loss = history.history['val_loss'][-1]

    return val_loss,

#### Определение процесса генетического алгоритма

In [68]:
def custom_mutation(individual, indpb):
    for i in range(len(individual)):
        if random.random() < indpb:
            if i == 0:  # n_layers (целое число)
                individual[i] = random.randint(1, 2)  # Максимум 2 слоя
            elif i == 1:  # n_neurons (целое число)
                individual[i] = random.randint(32, 128)  # От 32 до 128 нейронов
            elif i == 2:  # dropout_rate (вещественное число)
                individual[i] = random.uniform(0.0, 0.3)  # От 0.0 до 0.3
            elif i == 3:  # learning_rate (вещественное число)
                individual[i] = random.uniform(0.0005, 0.005)  # Уменьшенный диапазон
    return individual,

In [70]:
from joblib import Parallel, delayed

def optimize_model_with_ga(X_train, y_train, X_val, y_val, n_generations=3, population_size=5):
    # Определение индивидуала и функции приспособленности
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Минимизируем валидационную ошибку
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()

    # Определение атрибутов для каждого гиперпараметра
    toolbox.register("attr_n_layers", random.randint, 1, 2)
    toolbox.register("attr_n_neurons", random.randint, 32, 128)
    toolbox.register("attr_dropout_rate", random.uniform, 0.0, 0.3)
    toolbox.register("attr_learning_rate", random.uniform, 0.0005, 0.005)

    # Создание индивидуала
    toolbox.register("individual", tools.initCycle, creator.Individual,
                     (toolbox.attr_n_layers, toolbox.attr_n_neurons, toolbox.attr_dropout_rate, toolbox.attr_learning_rate), n=1)

    # Создание популяции
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Определение функции оценки
    def eval_function(individual):
        return evaluate_model(individual, X_train, y_train, X_val, y_val)

    toolbox.register("evaluate", eval_function)
    toolbox.register("mate", tools.cxUniform, indpb=0.5)
    toolbox.register("mutate", custom_mutation, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # Инициализация популяции
    pop = toolbox.population(n=population_size)

    # Использование Hall of Fame для отслеживания лучшего решения
    hof = tools.HallOfFame(1)

    # Параллельная оценка популяции
    def eval_population(population):
        fitnesses = Parallel(n_jobs=-1)(delayed(toolbox.evaluate)(ind) for ind in population)
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit

    # Основной цикл GA
    for gen in range(n_generations):
        print(f"Поколение {gen+1}/{n_generations}")

        # Оценка популяции
        invalid_ind = [ind for ind in pop if not ind.fitness.valid]
        if invalid_ind:
            eval_population(invalid_ind)

        # Обновление Hall of Fame
        hof.update(pop)

        # Применение операторов GA
        offspring = algorithms.varAnd(pop, toolbox, cxpb=0.5, mutpb=0.2)

        # Оценка потомков
        invalid_offspring = [ind for ind in offspring if not ind.fitness.valid]
        if invalid_offspring:
            eval_population(invalid_offspring)
        
        # Вывод лучшей валидационной ошибки в текущем поколении
        best = tools.selBest(pop, k=1)[0]
        print(f"Лучшая валидационная ошибка: {best.fitness.values[0]}")

        # Выбор новой популяции
        pop = toolbox.select(pop + offspring, k=len(pop))

    # Возвращаем лучший найденный индивидуал
    best_individual = hof[0]
    return best_individual

#### Обучение модели для каждого тикера

In [73]:
# Создание каталогов для сохранения моделей и графиков
if not os.path.exists('models'):
    os.makedirs('models')
if not os.path.exists('plots'):
    os.makedirs('plots')

# Цикл по каждому тикеру
for ticker in tickers:
    if not os.path.isfile(f'models/{ticker}_model.keras'):
        print(f"Обработка тикера: {ticker}")

        # Получение данных для тикера
        df_ticker = data[data['Ticker'] == ticker].sort_values('Date')

        # Используем 'Adj Close' в качестве целевой переменной
        close_prices = df_ticker['Adj Close'].values.reshape(-1, 1)

        # Проверка достаточности данных
        if len(close_prices) <= TIME_STEPS:
            print(f"Недостаточно данных для тикера {ticker}")
            continue

        # Масштабирование данных
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(close_prices)
        scalers[ticker] = scaler

        # Создание последовательностей
        X, y = create_sequences(scaled_data)

        # Проверка достаточности последовательностей
        if len(X) == 0:
            print(f"Недостаточно последовательностей для тикера {ticker}")
            continue

        # Разделение данных на обучающую и тестовую выборки (80% на обучение)
        split_index = int(0.8 * len(X))
        X_train_full, X_test = X[:split_index], X[split_index:]
        y_train_full, y_test = y[:split_index], y[split_index:]

        # Дополнительное разделение обучающих данных на обучение и валидацию (80% на обучение)
        val_split_index = int(0.8 * len(X_train_full))
        X_train, X_val = X_train_full[:val_split_index], X_train_full[val_split_index:]
        y_train, y_val = y_train_full[:val_split_index], y_train_full[val_split_index:]

        # Оптимизация модели с помощью GA
        best_hyperparams = optimize_model_with_ga(X_train, y_train, X_val, y_val, n_generations=3, population_size=5)

        # Извлечение лучших гиперпараметров
        n_layers = best_hyperparams[0]
        n_neurons = best_hyperparams[1]
        dropout_rate = best_hyperparams[2]
        learning_rate = best_hyperparams[3]

        # Построение финальной модели с лучшими гиперпараметрами
        K.clear_session()
        model = Sequential()
        model.add(Input(shape=(X_train_full.shape[1], X_train_full.shape[2])))

        if n_layers == 1:
            # Single LSTM layer
            model.add(LSTM(units=n_neurons, return_sequences=False))
            model.add(Dropout(dropout_rate))
        else:
            # First LSTM layer
            model.add(LSTM(units=n_neurons, return_sequences=True))
            model.add(Dropout(dropout_rate))
            # Middle LSTM layers
            for _ in range(n_layers - 2):
                model.add(LSTM(units=n_neurons, return_sequences=True))
                model.add(Dropout(dropout_rate))
            # Last LSTM layer
            model.add(LSTM(units=n_neurons, return_sequences=False))
            model.add(Dropout(dropout_rate))

        model.add(Dense(1))

        # Compile the model
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        model.compile(loss='mean_squared_error', optimizer=optimizer)

        # Обучение финальной модели на полной обучающей выборке
        model.fit(X_train_full, y_train_full, epochs=20, batch_size=32, verbose=0)

        # Оценка модели на тестовых данных
        y_pred = model.predict(X_test)
        y_pred_inverse = scaler.inverse_transform(y_pred)
        y_test_inverse = scaler.inverse_transform(y_test.reshape(-1, 1))

        # Расчет метрик оценки
        mse = mean_squared_error(y_test_inverse, y_pred_inverse)
        mae = mean_absolute_error(y_test_inverse, y_pred_inverse)

        performance[ticker] = {'MSE': mse, 'MAE': mae}

        # Сохранение модели
        model.save(f'models/{ticker}_model.keras')
        models[ticker] = model

        # Визуализация результатов
        plt.figure(figsize=(10, 6))
        plt.plot(y_test_inverse, label='Реальная цена')
        plt.plot(y_pred_inverse, label='Предсказанная цена')
        plt.title(f'Предсказание цены для {ticker}')
        plt.xlabel('Время')
        plt.ylabel('Цена')
        plt.legend()
        plt.savefig(f'plots/{ticker}_prediction.png')
        plt.close()
    else:
        print(f"Модель для тикера: {ticker} уже создана")

Обработка тикера: MMM
Поколение 1/3
Лучшая валидационная ошибка: 0.0012258400674909353
Поколение 2/3
Лучшая валидационная ошибка: 0.0012258400674909353
Поколение 3/3
Лучшая валидационная ошибка: 0.0012258400674909353
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Обработка тикера: AOS
Поколение 1/3
Лучшая валидационная ошибка: 0.0010215665679425001
Поколение 2/3
Лучшая валидационная ошибка: 0.001077264198102057
Поколение 3/3
Лучшая валидационная ошибка: 0.001077264198102057
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Обработка тикера: ABT
Поколение 1/3
Лучшая валидационная ошибка: 0.0023563208524137735
Поколение 2/3
Лучшая валидационная ошибка: 0.0023563208524137735
Поколение 3/3
Лучшая валидационная ошибка: 0.0017433736938983202
1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/stepWARNING:tensorflow:5 out of the last 11 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x34cc16020> triggered tf.function retracing. Tracing is expensive and the excessive number of tra

#### Сохранение волатильности и метрик производительности

In [80]:
for ticker in tickers:
    # Расчет доходностей для оценки волатильности
    returns = np.log(close_prices[1:] / close_prices[:-1])

    # Оценка волатильности (стандартное отклонение доходностей)
    volatility_value = np.std(returns)
    volatility[ticker] = volatility_value
    
# Преобразование словаря волатильности в DataFrame
volatility_df = pd.DataFrame.from_dict(volatility, orient='index', columns=['Volatility'])
volatility_df.to_csv('volatility.csv')

# Преобразование словаря производительности в DataFrame
performance_df = pd.DataFrame.from_dict(performance, orient='index')
performance_df.to_csv('model_performance.csv')

#### Организация и документирование результатов

In [ ]:
# Создание каталогов при необходимости (если не были созданы ранее)
if not os.path.exists('models'):
    os.makedirs('models')
if not os.path.exists('plots'):
    os.makedirs('plots')